# Лабораторная работа №4: Исследование случайного леса

## 1. Загрузка и подготовка данных

In [ ]:
import kagglehub
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, r2_score

# Загрузка и подготовка данных (повторяем шаги из лабы 1)

# Классификация
df_clf = pd.read_csv(kagglehub.dataset_download("ritesaluja/bank-note-authentication-uci-data") + "/BankNote_Authentication.csv")
X_clf = df_clf.drop('class', axis=1)
y_clf = df_clf['class']
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42)

# Регрессия
df_reg = pd.read_csv(kagglehub.dataset_download("mirichoi0218/insurance") + "/insurance.csv")
X_reg_full = df_reg.drop('charges', axis=1)
y_reg_full = df_reg['charges']
X_reg_full_train, X_reg_full_test, y_reg_full_train, y_reg_full_test = train_test_split(
    X_reg_full, y_reg_full, test_size=0.2, random_state=42)
X_reg = df_reg.drop(['sex', 'smoker', 'region', 'charges'], axis=1)
y_reg = df_reg['charges']
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42)

Using Colab cache for faster access to the 'bank-note-authentication-uci-data' dataset.
Using Colab cache for faster access to the 'insurance' dataset.


## 2. Создание бейзлайна и оценка качества

**Обучение модели**

In [ ]:
# ЗАДАЧА КЛАССИФИКАЦИИ
rf_clf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_clf.fit(X_clf_train, y_clf_train)

# ЗАДАЧА РЕГРЕССИИ
rf_reg = RandomForestRegressor(random_state=42, n_jobs=-1)
rf_reg.fit(X_reg_train, y_reg_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

**Делаем предсказания и считаем метрики**

In [ ]:
# КЛАССИФИКАЦИЯ
y_clf_pred_rf = rf_clf.predict(X_clf_test)
acc_baseline_rf = accuracy_score(y_clf_test, y_clf_pred_rf)
f1_baseline_rf = f1_score(y_clf_test, y_clf_pred_rf)

print("Классификация (Бейзлайн)")
print(f"Accuracy: {acc_baseline_rf:.4f}")
print(f"F1-score: {f1_baseline_rf:.4f}")

# РЕГРЕССИЯ
y_reg_pred_rf = rf_reg.predict(X_reg_test)
mae_baseline_rf = mean_absolute_error(y_reg_test, y_reg_pred_rf)
r2_baseline_rf = r2_score(y_reg_test, y_reg_pred_rf)

print("\nРегрессия (Бейзлайн)")
print(f"Mean Absolute Error: {mae_baseline_rf:.2f}")
print(f"R²: {r2_baseline_rf:.4f}")

Классификация (Бейзлайн)
Accuracy: 0.9927
F1-score: 0.9921

Регрессия (Бейзлайн)
Mean Absolute Error: 9278.82
R²: -0.0235


Мы получили для классификации высокие результаты, а для регрессии - очень плохие.

## 3. Улучшение бейзлайна

**Гипотезы для улучшения**

1.  **Гипотеза 1 (Общая):** Количество деревьев (`n_estimators`) по умолчанию (100) может быть неоптимальным. Изменение этого параметра может повлиять на качество и скорость обучения.
2.  **Гипотеза 2 (Общая):** Как и в случае с одним деревом, параметры, контролирующие сложность каждого дерева в лесу (`max_depth`, `min_samples_leaf`), являются ключевыми для настройки. Мы можем сделать деревья менее глубокими, чтобы еще больше снизить переобучение.
3.  **Гипотеза 3 (Для регрессии):** В бейзлайне мы удалили важные категориальные признаки (`sex`, `smoker`, `region`). Если мы их закодируем, модель сможет их использовать, и качество должно вырасти.

Проверим их.

In [ ]:
# ЗАДАЧА КЛАССИФИКАЦИИ (Гипотезы 1 и 2)

# Сетка параметров для подбора
param_grid_clf_rf = {
    'n_estimators': [100, 150],
    'max_depth': [10, 20, None],
    'min_samples_leaf': [1, 3],
    'max_features': [2, 3, None]
}

# Ищем лучшие параметры для дерева классификации
grid_search_clf_rf = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    param_grid_clf_rf,
    cv=3,
    scoring='f1'
)
grid_search_clf_rf.fit(X_clf_train, y_clf_train)

print(f"Лучшие параметры для классификации: {grid_search_clf_rf.best_params_}")
improved_model_clf_rf = grid_search_clf_rf.best_estimator_

Лучшие параметры для классификации: {'max_depth': 10, 'max_features': 2, 'min_samples_leaf': 1, 'n_estimators': 150}


In [ ]:
# ЗАДАЧА РЕГРЕССИИ (Гипотезы 1, 2, 3)

# Используем тот же препроцессор, что и в 3 лабе
numeric_features = ['age', 'bmi', 'children']
categorical_features = ['sex', 'smoker', 'region']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)])

# Создаем пайплайн
pipeline_reg_rf = Pipeline([
    ('preprocessor', preprocessor),
    ('rf', RandomForestRegressor(random_state=42, n_jobs=-1))
])

# Та же сетка параметров
param_grid_reg_rf = {
    'rf__n_estimators': [100, 150],
    'rf__max_depth': [5, 10, 15],
    'rf__min_samples_leaf': [1, 5, 10],
    'rf__max_features': [2, 3, None]
}

# Ищем лучшие параметры
grid_search_reg_rf = GridSearchCV(pipeline_reg_rf, param_grid_reg_rf, cv=3, scoring='r2')
grid_search_reg_rf.fit(X_reg_full_train, y_reg_full_train)

print(f"Лучшие параметры для регрессии: {grid_search_reg_rf.best_params_}")
improved_model_reg_rf = grid_search_reg_rf.best_estimator_

Лучшие параметры для регрессии: {'rf__max_depth': 5, 'rf__max_features': None, 'rf__min_samples_leaf': 10, 'rf__n_estimators': 100}


**Делаем предсказания на улучшенных моделях и считаем метрики**

In [ ]:
# Классификация
y_clf_pred_imp_rf = improved_model_clf_rf.predict(X_clf_test)
acc_improved_rf = accuracy_score(y_clf_test, y_clf_pred_imp_rf)
f1_improved_rf = f1_score(y_clf_test, y_clf_pred_imp_rf)

# Регрессия
y_reg_pred_imp_rf = improved_model_reg_rf.predict(X_reg_full_test)
mae_improved_rf = mean_absolute_error(y_reg_full_test, y_reg_pred_imp_rf)
r2_improved_rf = r2_score(y_reg_full_test, y_reg_pred_imp_rf)

# СРАВНЕНИЕ РЕЗУЛЬТАТОВ

print("Сравнение результатов")
print("Классификация:")
print(f"Accuracy: {acc_baseline_rf:.4f} -> {acc_improved_rf:.4f}")
print(f"F1-score: {f1_baseline_rf:.4f} -> {f1_improved_rf:.4f}")
print("\nРегрессия:")
print(f"MAE: {mae_baseline_rf:.2f} -> {mae_improved_rf:.2f}")
print(f"R²: {r2_baseline_rf:.4f} -> {r2_improved_rf:.4f}")

Сравнение результатов
Классификация:
Accuracy: 0.9927 -> 0.9927
F1-score: 0.9921 -> 0.9921

Регрессия:
MAE: 9278.82 -> 2488.78
R²: -0.0235 -> 0.8774


**Выводы по улучшению**

**Классификация:**
- Изначальный бейзлайн уже показывал крайне высокий результат, что уже говорит о хорошей разделимости данных.
- После подбора гиперпараметров, ограничивающих сложность дерева (`max_depth`, `min_samples_leaf`, `n_estimators`) качество предсказаний не изменилось.

**Регрессия:**
- Бейзлайн был крайне слабым, так как не использовал категориальные признаки и не ограничивал глубину дерева.
- После применения пайплайна кодированием категориальных и ограничения глубины дерева, качество модели выросло. R² стал положительным и довольно высоким, а MAE значительно снизилась.


Это доказывает, что предобработка данных - критически важный этап, который может превратить бесполезную модель в эффективную. Все гипотезы подтвердились.

## 4.	Имплементация алгоритма машинного обучения

In [ ]:
from scipy import stats

# Базовый класс
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, *, value=None):
        self.feature_index = feature_index # Индекс признака для разделения
        self.threshold = threshold         # Пороговое значение
        self.left = left                   # Левый дочерний узел
        self.right = right                 # Правый дочерний узел
        self.value = value                 # Значение в листе (класс или число)

class MyDecisionTree:
    # Базовый класс для дерева
    def __init__(self, min_samples_split=2, max_depth=100, max_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None
        self.max_features = max_features

    def fit(self, X, y):
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # Условия остановки рекурсии
        if (depth >= self.max_depth or
            n_labels == 1 or
            n_samples < self.min_samples_split):
            leaf_value = self._leaf_value(y)
            return Node(value=leaf_value)

        if self.max_features is None:
            n_feats_to_use = n_features
        else:
            n_feats_to_use = min(self.max_features, n_features)

        feat_idxs = np.random.choice(n_features, n_feats_to_use, replace=False)
        best_feat, best_thresh = self._best_split(X, y, feat_idxs)

        # Если не удалось найти разделение, создаем лист
        if best_feat is None:
            leaf_value = self._leaf_value(y)
            return Node(value=leaf_value)

        # Рекурсивно строим дочерние деревья
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feat, best_thresh, left, right)

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold
        return split_idx, split_thresh

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature_index] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    # Эти методы будут переопределены в дочерних классах
    def _information_gain(self, y, X_column, split_thresh):
        raise NotImplementedError

    def _leaf_value(self, y):
        raise NotImplementedError

class MyDecisionTreeClassifier(MyDecisionTree):
    # Класс для классификации
    def _gini(self, y):
        if len(y) == 0: return 0
        _, counts = np.unique(y, return_counts=True)
        probs = counts / len(y)
        return 1 - np.sum(probs**2)

    def _information_gain(self, y, X_column, split_thresh):
        parent_gini = self._gini(y)
        left_idxs, right_idxs = self._split(X_column, split_thresh)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        gini_l, gini_r = self._gini(y[left_idxs]), self._gini(y[right_idxs])
        child_gini = (n_l / n) * gini_l + (n_r / n) * gini_r
        ig = parent_gini - child_gini
        return ig

    def _leaf_value(self, y):
        from collections import Counter
        most_common = Counter(y).most_common(1)
        return most_common[0][0] if most_common else None

class MyDecisionTreeRegressor(MyDecisionTree):
    # Класс для регрессии
    def _mse(self, y):
        if len(y) == 0: return 0
        mean = np.mean(y)
        return np.mean((y - mean)**2)

    def _information_gain(self, y, X_column, split_thresh):
        # В регрессии мы максимизируем уменьшение дисперсии (MSE)
        parent_mse = self._mse(y)
        left_idxs, right_idxs = self._split(X_column, split_thresh)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        mse_l, mse_r = self._mse(y[left_idxs]), self._mse(y[right_idxs])
        child_mse = (n_l / n) * mse_l + (n_r / n) * mse_r
        ig = parent_mse - child_mse
        return ig

    def _leaf_value(self, y):
        if len(y) == 0:
            return 0
        return np.mean(y)

class MyRandomForest:
    def __init__(self, base_tree_class, n_estimators=100, max_depth=10, min_samples_split=2, max_features=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.base_tree_class = base_tree_class
        self.trees = []
        self.max_features = max_features

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_estimators):
            tree = self.base_tree_class(
                min_samples_split=self.min_samples_split,
                max_depth=self.max_depth,
                max_features=self.max_features
            )
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(predictions, 0, 1)
        return self._combine_predictions(tree_preds)

    def _combine_predictions(self, tree_preds):
        raise NotImplementedError

class MyRandomForestClassifier(MyRandomForest):
    def __init__(self, **kwargs):
        super().__init__(base_tree_class=MyDecisionTreeClassifier, **kwargs)

    def _combine_predictions(self, tree_preds):
        # Голосование: для каждого сэмпла находим самое частое значение
        y_pred = [stats.mode(pred, keepdims=False)[0] for pred in tree_preds]
        return np.array(y_pred)

class MyRandomForestRegressor(MyRandomForest):
    def __init__(self, **kwargs):
        super().__init__(base_tree_class=MyDecisionTreeRegressor, **kwargs)

    def _combine_predictions(self, tree_preds):
        # Усреднение: для каждого сэмпла считаем среднее
        return np.mean(tree_preds, axis=1)

# Подготовим numpy-массивы для нашей реализации
X_clf_train_np = X_clf_train.to_numpy()
y_clf_train_np = y_clf_train.to_numpy()
X_clf_test_np = X_clf_test.to_numpy()

X_reg_train_np = X_reg_train.to_numpy()
y_reg_train_np = y_reg_train.to_numpy()
X_reg_test_np = X_reg_test.to_numpy()

In [ ]:
# КЛАССИФИКАЦИЯ (сравнение с бейзлайном)
my_rf_clf = MyRandomForestClassifier(n_estimators=20, max_depth=10, max_features=None)
my_rf_clf.fit(X_clf_train_np, y_clf_train_np)
my_y_clf_pred = my_rf_clf.predict(X_clf_test_np)

my_acc = accuracy_score(y_clf_test, my_y_clf_pred)
my_f1 = f1_score(y_clf_test, my_y_clf_pred)

print("Своя реализация vs Sklearn (Бейзлайн)")
print("Классификация:")
print(f"Accuracy: {acc_baseline_rf:.4f} (sklearn) -> {my_acc:.4f} (своя)")
print(f"F1-score: {f1_baseline_rf:.4f} (sklearn) -> {my_f1:.4f} (своя)")

# РЕГРЕССИЯ (сравнение с бейзлайном)
my_rf_reg = MyRandomForestRegressor(n_estimators=20, max_depth=10, max_features=None)
my_rf_reg.fit(X_reg_train_np, y_reg_train_np)
my_y_reg_pred = my_rf_reg.predict(X_reg_test_np)

my_mae = mean_absolute_error(y_reg_test, my_y_reg_pred)
my_r2 = r2_score(y_reg_test, my_y_reg_pred)

print("\nРегрессия:")
print(f"MAE: {mae_baseline_rf:.2f} (sklearn) -> {my_mae:.2f} (своя)")
print(f"R²: {r2_baseline_rf:.4f} (sklearn) -> {my_r2:.4f} (своя)")

Своя реализация vs Sklearn (Бейзлайн)
Классификация:
Accuracy: 0.9927 (sklearn) -> 0.9855 (своя)
F1-score: 0.9921 (sklearn) -> 0.9840 (своя)

Регрессия:
MAE: 9278.82 (sklearn) -> 9113.18 (своя)
R²: -0.0235 (sklearn) -> 0.0556 (своя)


**Выводы по своей реализации (сравнение с бейзлайном)**

Моя реализация случайного леса показала результаты, сопоставимые с бейзлайном `sklearn` на обоих наборах данных. Это подтверждает, что базовая логика бэггинга и агрегации предсказаний работает корректно.

In [ ]:
# КЛАССИФИКАЦИЯ (с улучшениями)
best_params_clf_rf = grid_search_clf_rf.best_params_
my_rf_clf_imp = MyRandomForestClassifier(
    n_estimators=best_params_clf_rf['n_estimators'],
    max_depth=best_params_clf_rf['max_depth'],
    min_samples_split=best_params_clf_rf['min_samples_leaf'],
    max_features=best_params_clf_rf.get('max_features')
)
my_rf_clf_imp.fit(X_clf_train_np, y_clf_train_np)
my_y_clf_pred_imp = my_rf_clf_imp.predict(X_clf_test_np)

my_acc_imp = accuracy_score(y_clf_test, my_y_clf_pred_imp)
my_f1_imp = f1_score(y_clf_test, my_y_clf_pred_imp)

# РЕГРЕССИЯ (с улучшениями)
preprocessor_reg = improved_model_reg_rf.named_steps['preprocessor']
X_reg_train_processed = preprocessor_reg.transform(X_reg_full_train)
X_reg_test_processed = preprocessor_reg.transform(X_reg_full_test)
y_reg_train_processed = y_reg_full_train.to_numpy()

best_params_reg_rf = grid_search_reg_rf.best_params_
my_rf_reg_imp = MyRandomForestRegressor(
    n_estimators=best_params_reg_rf['rf__n_estimators'],
    max_depth=best_params_reg_rf['rf__max_depth'],
    min_samples_split=best_params_reg_rf['rf__min_samples_leaf'],
    max_features=best_params_reg_rf.get('rf__max_features')
)
my_rf_reg_imp.fit(X_reg_train_processed, y_reg_train_processed)
my_y_reg_pred_imp = my_rf_reg_imp.predict(X_reg_test_processed)

my_mae_imp = mean_absolute_error(y_reg_full_test, my_y_reg_pred_imp)
my_r2_imp = r2_score(y_reg_full_test, my_y_reg_pred_imp)

# ИТОГОВОЕ СРАВНЕНИЕ

print("Своя реализация vs Sklearn (Улучшенные)")
print("Классификация:")
print(f"Accuracy: {acc_improved_rf:.4f} (sklearn) -> {my_acc_imp:.4f} (своя)")
print(f"F1-score: {f1_improved_rf:.4f} (sklearn) -> {my_f1_imp:.4f} (своя)")
print("\nРегрессия:")
print(f"MAE: {mae_improved_rf:.2f} (sklearn) -> {my_mae_imp:.2f} (своя)")
print(f"R²: {r2_improved_rf:.4f} (sklearn) -> {my_r2_imp:.4f} (своя)")

Своя реализация vs Sklearn (Улучшенные)
Классификация:
Accuracy: 0.9927 (sklearn) -> 0.9891 (своя)
F1-score: 0.9921 (sklearn) -> 0.9880 (своя)

Регрессия:
MAE: 2488.78 (sklearn) -> 2528.77 (своя)
R²: 0.8774 (sklearn) -> 0.8692 (своя)


### Выводы по своей реализации (сравнение с улучшенной моделью)

После применения тех же техник предобработки, моя реализация случайного леса показала результаты, очень близкие к улучшенным моделям из `sklearn`. Это доказывает, что моя реализация корректно использует гиперпараметры для контроля сложности дерева. Для задачи регрессии, после подачи на вход обработанных данных (с закодированными категориями), качество моей модели также значительно возросло.

**Общий вывод по лабораторной работе №4:**
1.  Случайный лес - это мощный и универсальный ансамблевый метод, который, как правило, превосходит по качеству одиночное решающее дерево за счет уменьшения дисперсии и устойчивости к переобучению.
2.  Ключевыми идеями, лежащими в основе его успеха, являются бэггинг (обучение на случайных подвыборках) и метод случайных подпространств (использование случайного подмножества признаков).